### What is non-linear regression?
Regression model : $y = \mu(x)+\epsilon$<br>
Linear model : $y = \beta_0 + \beta_1 x_1 + \beta_2x_2+...+\epsilon$<br>
Non-linear model : if $\frac{\delta \mu(x)}{\delta \beta}$ is function about $\beta$, then it is non-linear model<br>
ex) $\mu(x)=e^{\beta_0+\beta_1x_1},\ \frac{\delta \mu(x)}{\delta \beta}=[e^{\beta_0+\beta_1x_1}, \ x_1e^{\beta_0+\beta_1x_1}]$

### How to get parameter in non-linear regression?

**1. Transformation**<br>
$y=e^{\beta_0+\beta_1x_1}+\epsilon$, we can transform that equation to $lny = y' = \beta_0+\beta_1x_1+\epsilon '$ and can solve by OLS<br>

**2. Optimization**<br>
If you can't use transformation method because of difficult case such as $$y = exp(-\beta_1x_1exp(-\beta_2(\frac{1}{x_2}-\frac{1}{100}))+\epsilon$$
then we can find $\beta$ by optimization method such as newton-raphson method, gauss-newton method, ... etc


### Gauss-newton method
https://en.wikipedia.org/wiki/Gauss%E2%80%93Newton_algorithm
- Very simple algorithm


1. Set initial $\beta=\beta^{(0)}$
2. Get next $\beta$ by  $\beta^{(1)} = \beta^{(0)} - [(\frac{\delta e(\beta^{(0)})}{\delta \beta})^T
(\frac{\delta e(\beta^{(0)})}{\delta \beta})]^{-1}[(\frac{\delta e(\beta^{(0)})}{\delta \beta})^Te(\beta^{(0)})]$, and $e(\beta) = y-\mu(x)$
3. Update $\beta$ and repeat until $\beta^{(t)} - \beta^{(t-1)} < s$(very small value)

### Start
**simple case**<br>
$y=exp(\beta_0+\beta_1x)+\epsilon$<br>
and answer : $(\beta_0,\beta_1) = (5,2)$

In [1]:
import numpy as np

In [2]:
np.random.seed(2020)
x = np.random.randn(100)
y = np.exp(5+2*x) + np.random.normal(0,1,100)


In [201]:
#mu
def mu(beta, x):
    return np.exp(beta[0]+beta[1]*x)

In [138]:
#Differential function
#Result is nx2 matrix (because the number of beta is 2)
def diff(func, beta, x, h = 0.001):

    result = []
    for i in range(len(beta)):
        betaH = beta.copy()
        betaH=betaH.astype('float32')
        betaH[i] = beta[i]+h
        
        #why -1 : e(x) = y - mu(x)
        result.append(-1*(func(betaH,x)-func(beta,x))/h)
                      
    return np.array(result).T

In [160]:
#Update and repeat
def updateRepeat(func, beta):
    try:
        for i in range(1000):
            bP = diff(mu,beta,x)
            e = y-mu(beta,x)

            betaNext = beta - np.linalg.inv(bP.T @ bP) @ bP.T @ e
            if(sum(abs(betaNext-beta)) < 0.0001):
                break
            beta = betaNext
            print(beta)
        return beta
    except:
        return -1
    

In [165]:
#initial beta
beta = np.array([4,1])

updateRepeat(mu,beta)

[-1.19168502 14.16088787]
[-2.19120214 14.16074116]
[-3.19080303 14.16043774]
[-4.19039219 14.16028568]
[-5.18979172 14.16055212]
[-6.18932269 14.16045869]
[-7.18867624 14.16033612]
[-8.18834586 14.16070586]
[-9.18745762 14.1603986 ]
[-10.1868716   14.16023927]
[-11.1856117   14.16007953]
[-12.18395492  14.16019274]
[-13.18121403  14.15910149]
[-14.17487729  14.15680898]
[-15.1592374   14.15047627]
[-16.11660376  14.13379185]
[-17.00285776  14.08842332]
[-17.69873266  13.96747401]
[-17.91456353  13.65466837]
[-17.04665969  12.90817808]
[-14.31613818  11.41884615]
[-9.86706059  9.24721329]
[-4.97594412  6.90607654]
[-0.40834452  4.72151608]
[3.06094605 3.03342645]
[4.67467254 2.19677418]
[4.9808169  2.01279777]
[4.99986867 2.00011901]
[5.00000591 2.00002252]


array([5.00000591, 2.00002252])

Find answer!

In [166]:
#initial beta
beta = np.array([3,1])

updateRepeat(mu,beta)

[-9.39491566 36.77500233]
[-10.1298496  36.7587958]
[-11.23886765  36.74969545]
[-12.30539654  36.76643791]
[-13.25877818  36.7794975 ]
[-14.36471556  36.76392119]
[-16.01803565  36.87788353]
[-16.91260695  36.86149556]
[-18.80007354  37.2017385 ]
[-20.14788809  37.3241266 ]
[-21.08439037  37.31912348]
[-22.06145243  37.26671068]
[-23.5574193   37.44528325]
[-26.17133251  37.76077414]
[-28.43506496  37.75894557]


-1

Fail to find

### Second case
$$y=\frac{\beta_1}{x_1+0.5} + \frac{\beta_4}{exp(0.5+\beta_2x_2+\frac{1}{\beta_3+x_3})}+\epsilon$$
answer : $(\beta_1,\beta_2,\beta_3,\beta_4) = (3,2,7,9)$

In [206]:
np.random.seed(2020)
x1 = np.random.normal(5,10,100)
x2 = np.random.normal(0,1,100)
x3 = np.random.normal(3,5,100)

x=[x1,x2,x3]
y = 3/(x1+0.5) + 9/np.exp(0.5+2*x2 + 1/(7+x3))


In [207]:
#mu
def mu(beta, x):
    return beta[0]/(x[0]+0.5) + beta[3]/np.exp(0.5+beta[1]*x[1] + 1/(beta[2]+x[2]))

In [224]:
#initial beta
beta = np.array([1,1,8,20])

updateRepeat(mu,beta)

[  3.74105671   3.09640227   8.18573973 -11.00856146]
[3.05228786 3.18889896 8.20364609 3.08108231]
[3.05946941 2.90038412 8.16266192 2.67334849]
[3.03642934 2.42861619 8.06474619 4.08144112]
[3.00943785 2.03127982 7.8635932  7.09060376]
[3.00178127 1.99477668 7.5644919  8.89878526]
[3.00181213 1.99954245 7.20825694 8.98050046]
[3.00032139 1.99993389 7.00663296 8.99974273]
[3.00000024 1.9999995  6.99999433 8.99999994]


array([3.00000024, 1.9999995 , 6.99999433, 8.99999994])

### Result
We can find answer but **initial beta is very very important**